In [12]:
"""Code assumes the ability to train using a GPU with CUDA.
"""
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from advertorch.attacks import GradientSignAttack, CarliniWagnerL2Attack, PGDAttack
import matplotlib.pyplot as plt
import gc

# makes default tensor a CUDA tensor so GPU can be used
torch.set_default_tensor_type('torch.cuda.FloatTensor')

### Define data loaders and data preprocessing steps

In [13]:
data_preprocess = torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.1307,), (0.3081,))])
# the mean of mnist pixel data is .1307 and the stddev is .3081

train_loader = torch.utils.data.DataLoader(
                    torchvision.datasets.MNIST('./data', train=True, download=True,
                         transform=data_preprocess), 
                    batch_size=64, 
                    shuffle=False)

test_loader = torch.utils.data.DataLoader(
                    torchvision.datasets.MNIST('./data', train=False, download=True,
                         transform=data_preprocess), 
                    batch_size=50, 
                    shuffle=False)

### Define the model

In [14]:
class LeNet(nn.Module):
    """MNIST-modified LeNet-5 model.
    """
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc1_drop = nn.Dropout(p=.50)
        self.fc2 = nn.Linear(120, 84)
        self.fc2_drop = nn.Dropout(p=.50)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(x.shape[0], -1)
        x = self.fc1_drop(F.relu(self.fc1(x)))
        x = self.fc2_drop(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

### Define adversarial example generation function

In [15]:
def generate_adversarial_samples(og_samples, true_labels, adversary, num_per_samp=1):
    """Create num_per_samp adversarial examples for each sample in
    og_samples. Return the generated samples along with corresponding 
    adv_labels, a tensor containing the adversarial examples' labels.
    """
    adv_samples = []
    for i in range(num_per_samp):
        adv_samples.append(adversary.perturb(og_samples, true_labels))
    adv_samples = torch.cat(adv_samples, 0)
    adv_labels = torch.cat([true_labels]*num_per_samp, 0)
    return adv_samples, adv_labels

### Define my loss function


In [16]:
def my_loss(output, labels, alpha_wd=0, alpha_jr=0, x=None, bp_mat=None):
    """Adds terms for L2-regularization and the norm of the input-output 
    Jacobian to the standard cross-entropy loss function. Check https://arxiv.org/abs/1908.02729
    for alpha_wd, alpha_jr suggestions.
    """
    # standard cross-entropy loss base
    loss = F.cross_entropy(output, labels)
    
    # add l2 regularization to loss 
    if alpha_wd != 0:
        l2 = 0
        for p in lenet.parameters():
            l2 += p.pow(2).sum()
        loss = loss + alpha_wd*l2
    
    # add input-output jacobian regularization formulation
    if alpha_jr != 0:
        n_outputs = output.shape[1]
        batch_size = x.shape[0]
        # needed because some edge-case batches are not standard size
        if bp_mat.shape[0]/n_outputs != batch_size:     
            bp_mat = bp_matrix(batch_size, n_outputs)
        x = x.repeat(n_outputs, 1, 1, 1)
        # needed so that we can get gradient of output w.r.t input
        x = Variable(x, requires_grad=True)
        y = lenet(x)
        x_grad = torch.autograd.grad(y, x, grad_outputs=bp_mat, create_graph=True)[0]
        # get sum of squared values of the gradient values 
        j = x_grad.pow(2).sum() / batch_size
        loss = loss + alpha_jr*j
        # needed so gradients don't accumulate in leaf variables when calling loss.backward in train function
        optimizer.zero_grad()

    return loss

### Functions for Jacobian regularization

In [17]:
def avg_norm_jacobian(x, net, n_outputs):
    """Returns squared frobenius norm of the input-output Jacobian averaged 
    over the entire batch of inputs in x.
    """
    batch_size = x.shape[0]
    x = x.repeat(n_outputs, 1, 1, 1)
    x = Variable(x, requires_grad=True)
    # needed so that we can get gradient of output w.r.t input
    y = net(x)
    x_grad = torch.autograd.grad(y, x, grad_outputs=bp_mat)[0]
    j = x_grad.pow(2).sum() / batch_size
    return j

In [ ]:
def bp_matrix(batch_size, n_outputs):
    """Creates matrix that is used to calculate Jacobian for multiple input 
    samples at once.
    """
    idx = torch.arange(n_outputs).reshape(n_outputs,1).repeat(1,batch_size).reshape(batch_size*n_outputs,)
    return torch.zeros(len(idx), n_outputs).scatter_(1, idx.unsqueeze(1), 1.)

### Define train and test functions 

In [18]:
def train(alpha_wd, alpha_jr, adversary=None):
    lenet.train()
    
    for batch_idx, (samples, labels) in enumerate(train_loader):
        # sends to GPU, i.e. essentially converts from torch.FloatTensor to torch.cuda.FloatTensor
        samples, labels = samples.to(device), labels.to(device)
        
        # expand dataset with adversarial examples if adversary specified
        if adversary != None:
            adv_samples, adv_labels = generate_adversarial_samples(samples, labels, adversary)  
            samples, labels = torch.cat([samples, adv_samples], 0), torch.cat([labels, adv_labels], 0)
                
        optimizer.zero_grad()
        
        output = lenet(samples)
        
        loss = my_loss(output, labels, alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples, bp_mat=tr)
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            j = avg_norm_jacobian(samples, lenet, output.shape[1])
            print(f'\tLoss: {loss.item():.6f} Average norm of Jacobian: {j:6f}')
            train_losses.append(loss.item())

In [19]:
def test(alpha_wd, alpha_jr):
    lenet.eval()
    test_loss = 0
    correct = 0
    
    for samples, labels in test_loader:
        samples, labels = samples.to(device), labels.to(device)
        output = lenet(samples)
        test_loss += my_loss(output, labels, alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples, bp_mat=te).item()
        # output is a tensor, .data retrieves its data, max returns the index of the highest valued element
        preds = output.data.max(1, keepdim=True)[1]
        correct += preds.eq(labels.data.view_as(preds)).sum().item()
                
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * float(correct / len(test_loader.dataset))
    
    print(f'\tTest set accuracy: ({test_accuracy:.2f}%)')
    
    test_accuracies.append(test_accuracy)
    test_losses.append(test_loss)

### Training

In [22]:
# training details
torch.manual_seed(1)
n_epochs = 30
log_interval = 200
training_round = 1

# varying values for certain hyperparameters to produce models with varying degrees of robustness
epsilons = [0, .1, .2, .3, .4, .5, .6, .7]
alpha_wds = [0.00005, .000001, .000005, .00001, .00005, .0001, .0005, .001]
alpha_jrs = [.00000001, .0000001, .000001, .00001, .0001, .001, .01, .1]

# these needed so that calculating jacobian across a batch of inputs is parallelizable
tr = bp_matrix(64, 10)
te = bp_matrix(50, 10)

# dictionary to record each model's training/testing stats
performance = {}

for hyp_param_to_vary in [alpha_jrs, epsilons, alpha_wds]:
    epsilon = 0
    alpha_wd = 0
    alpha_jr = 0
    
    for value in hyp_param_to_vary:        
        # change hyperparameter that is being varied
        if hyp_param_to_vary == epsilons:
            epsilon = value
        elif hyp_param_to_vary == alpha_wds:
            alpha_wd = value
        else:
            alpha_jr = value
        
        print(f'\nBeginning training for model: models/ep{epsilon}_wd{alpha_wd}_jr{alpha_jr}_{training_round}')

        # instantiate model and optimizer
        learning_rate = 0.01
        momentum = 0.9
        lenet = LeNet()
        optimizer = optim.SGD(lenet.parameters(), lr=learning_rate, momentum=momentum)
        lr_decayer = StepLR(optimizer, step_size=10, gamma=0.1)

        # make model CUDA enabled and define GPU/device to use
        lenet.cuda()
        device = 0
        
        # define adversary to train against if needed
        FGSM = None
        if epsilon != 0:
            FGSM = GradientSignAttack(predict=lenet, loss_fn=F.cross_entropy, 
                            eps=epsilon, clip_min=-3., clip_max=3., targeted=False)

        # for tracking training progress
        train_losses = []
        test_losses = []
        test_accuracies = []

        for epoch in range(1, n_epochs + 1):
            print(f'Epoch #{epoch}')
            train(alpha_wd=alpha_wd, alpha_jr=alpha_jr, adversary=FGSM)
            test(alpha_wd=alpha_wd, alpha_jr=alpha_jr)
            lr_decayer.step()
        
        performance[f'ep{epsilon}_wd{alpha_wd}_jr{alpha_jr}'] = (train_losses, test_losses, test_accuracies)
        torch.save(lenet.state_dict(), f'models/ep{epsilon}_wd{alpha_wd}_jr{alpha_jr}')


Beginning training for model: models/ep0_wd0_jr0.01_1
Epoch #1
	Loss: 2.300334 Average norm of Jacobian: 0.021065
	Loss: 0.600776 Average norm of Jacobian: 6.331427
	Loss: 0.241412 Average norm of Jacobian: 7.554722
	Loss: 0.331851 Average norm of Jacobian: 7.947834
	Loss: 0.637592 Average norm of Jacobian: 6.903455
	Test set accuracy: (96.69%)
Epoch #2
	Loss: 0.298643 Average norm of Jacobian: 6.309241
	Loss: 0.193557 Average norm of Jacobian: 5.886839
	Loss: 0.172586 Average norm of Jacobian: 6.344795
	Loss: 0.223054 Average norm of Jacobian: 6.685603
	Loss: 0.315696 Average norm of Jacobian: 6.273883
	Test set accuracy: (97.65%)
Epoch #3
	Loss: 0.198904 Average norm of Jacobian: 5.357544
	Loss: 0.196035 Average norm of Jacobian: 5.312222
	Loss: 0.127478 Average norm of Jacobian: 5.023313
	Loss: 0.219904 Average norm of Jacobian: 5.885437
	Loss: 0.447362 Average norm of Jacobian: 5.653174
	Test set accuracy: (97.66%)
Epoch #4
	Loss: 0.165446 Average norm of Jacobian: 5.378593
	Loss:

KeyboardInterrupt: 

### Write performance dictionary to text file

In [28]:
f = open(f'models/training_round_{training_round}_performance.txt','w')
f.write(str(performance))
f.close()

# to read dictionary from file:
# f = open(f'models/training_round_{training_round}_performance.txt','r')
# d = eval(f.read())